## The Factory Pattern
In this pattern we ask for an object without knowing where the object is coming from (that is, which class is used to generate it). The idea behind a factory is to simplify an object creation. There are two forms:
* The Factory Method returns a different object per input parameter
* The Abstracct Factory is a group of Factory Methods used to create a family of related products.

Tracking your objects also becomes a lot easier when object creation is centralized.

In [1]:
help(id)

Help on built-in function id in module __builtin__:

id(...)
    id(object) -> integer
    
    Return the identity of an object.  This is guaranteed to be unique among
    simultaneously existing objects.  (Hint: it's the object's memory address.)



In [2]:
x = 2
id(x)

31690192L

The fact that the memory addresses are different means that two distinct objects are created as follows:

In [3]:
class A(object):
    pass

if __name__ == '__main__':
    a = A()
    b = A()
    
print(id(a) == id(b))
print(a, b)

False
(<__main__.A object at 0x0000000003DDA2B0>, <__main__.A object at 0x0000000003DDA2E8>)


## An example of Factory Method with XML and JSON

In [4]:
import json

class JSONConnector:
    ''' parses the json file. 
        Results of json.load are held in the self.data attribute 
        These results can be collected from the parsed_data property '''
    def __init__(self, filepath):
        self.data = dict()
        with open(filepath, mode='r') as f:
            self.data = json.load(f)
            
    @property
    def parsed_data(self):
        return self.data


In [21]:
import xml.etree.ElementTree as etree
import xmltodict

class XMLConnector:
    ''' similar setup to JSONConnector class, but here we're reading XML files '''
    def __init__(self, filepath):
        with open(filepath) as f:
            self.data = xmltodict.parse(f.read())

    @property
    def parsed_data(self):
        return self.data["root"]


In [6]:
def connection_factory(filepath):
    ''' returns a connector object depending on the file extension '''
    if filepath.endswith('json'):
        connector = JSONConnector
    elif filepath.endswith('xml'):
        connector = XMLConnector
    else:
        raise ValueError('Cannot connect to {}'.format(filepath))
    return connector(filepath)


In [7]:
def connect_to(filepath):
    ''' wrapper to connection_factory adding exception handling '''
    factory = None
    try:
        factory = connection_factory(filepath)
    except ValueError as ve:
        print(ve)
    return factory

In [8]:
sqlite_factory = connect_to('data/person.sq3') # connect_to function prints the value error raised in connection_factory

Cannot connect to data/person.sq3


In [9]:
! pip install dicttoxml

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
! pip install xmltodict

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [12]:
# here we create some test files as input
from dicttoxml import dicttoxml
import xmltodict
import os

myDict = {
    'time': {"hour":"1", "minute":"30","seconds": "40"},
    'place': {"street":"40 something", "zip": "00000"}
}

xml = dicttoxml(myDict)

# kill the test files if they already exist
for f in ['testXmlFile.xml', 'testJsonFile.json']:
    if os.path.isfile(f):
        os.remove(f)

with open('testXmlFile.xml', 'w') as f:
    f.write(xml)
    
with open('testJsonFile.json', 'w') as f:
    json.dump(myDict, f)

Using the connect_to function to create connection objects:

In [22]:
obj1 = connect_to('testXmlFile.xml')
obj2 = connect_to('testJsonFile.json')

In [23]:
obj1

<__main__.XMLConnector instance at 0x0000000003F80E88>

In [24]:
obj2

<__main__.JSONConnector instance at 0x0000000003F9DDC8>

In [25]:
dict(obj1.parsed_data)

{u'place': OrderedDict([(u'@type', u'dict'),
              (u'street',
               OrderedDict([(u'@type', u'str'), ('#text', u'40 something')])),
              (u'zip',
               OrderedDict([(u'@type', u'str'), ('#text', u'00000')]))]),
 u'time': OrderedDict([(u'@type', u'dict'),
              (u'seconds',
               OrderedDict([(u'@type', u'str'), ('#text', u'40')])),
              (u'minute', OrderedDict([(u'@type', u'str'), ('#text', u'30')])),
              (u'hour', OrderedDict([(u'@type', u'str'), ('#text', u'1')]))])}

In [26]:
obj2.parsed_data

{u'place': {u'street': u'40 something', u'zip': u'00000'},
 u'time': {u'hour': u'1', u'minute': u'30', u'seconds': u'40'}}

In [40]:
obj1.parsed_data["place"]["street"]["#text"]

u'40 something'

In [41]:
obj2.parsed_data["place"]["street"]

u'40 something'

## The Abstract Factory Pattern
An Abstract Factory is a (logical) group of Factory Methods, where each Factory Method is responsible for generating a different kind of object

In [42]:
class Frog:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name
    
    def interact_with(self, obstacle):
        print('{} the Frog encounters {} and {}!'.format(self, obstacle, obstacle.action()))


In [43]:
class Bug:
    def __str__(self):
        return 'a bug'
    
    def action(self):
        return 'eats it'


In [44]:
class FrogWorld:
    ''' The FrogWorld class is an Abstract Factory. Its main responsibilities are creating
        the main character and the obstacle(s) of the game. '''
    
    def __init__(self, name):
        print(self)
        self.player_name = name
        
    def __str__(self):
        return '\n\n\t------ Frog World -------'
    
    def make_character(self):
        return Frog(self.player_name)

    def make_obstacle(self):
        return Bug()
    

In [45]:
class Wizard:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name
    
    def interact_with(self, obstacle):
        print('{} the Wizard battles against {} and {}!'.format(self, obstacle, obstacle.action()))
        

In [46]:
class Ork:
    def __str__(self):
        return 'an evil ork'
    
    def action(self):
        return 'kills it'


In [47]:
class WizardWorld:
    ''' The WizardWorld class is an Abstract Factory. Its main responsibilities are creating
        the main character and the obstacle(s) of the game. '''

    def __init__(self, name):
        print(self)
        self.player_name = name
        
    def __str__(self):
        return '\n\n\t------ Wizard World -------'
    
    def make_character(self):
        return Wizard(self.player_name)
    
    def make_obstacle(self):
        return Ork()


In [48]:
class GameEnvironment:
    
    def __init__(self, factory):
        self.hero = factory.make_character()
        self.obstacle = factory.make_obstacle()
        
    def play(self):
        self.hero.interact_with(self.obstacle)


In [57]:
def validate_age(name):
    try:
        age = raw_input('Welcome {}. How old are you? '.format(name))
        age = int(age)
    except ValueError as err:
        print("Age {} is invalid, please try again...".format(age))
        return (False, age)
    return (True, age)


In [58]:
def main():
    name = raw_input("Hello. What's your name? ")
    valid_input = False
    while not valid_input:
        valid_input, age = validate_age(name)
    game = FrogWorld if age < 18 else WizardWorld
    environment = GameEnvironment(game(name))
    environment.play()


In [59]:
main() # kick things off

Hello. What's your name? Simon
Welcome Simon. How old are you? 17


	------ Frog World -------
Simon the Frog encounters a bug and eats it!


In [60]:
main()

Hello. What's your name? Simon
Welcome Simon. How old are you? 18


	------ Wizard World -------
Simon the Wizard battles against an evil ork and kills it!
